# Loading and disaplying the 3DMM

In [ ]:
# import packages
import pickle
import numpy as np
import trimesh
import networkx as nx
import pandas as pd
import time
import multiprocessing as mp
import os
from datetime import datetime
from Prediction_functions import *

## Original model including everything

In [ ]:
# load model
models_folder = './datasets/TMS/3DMM/UHM_models/'
currnet_model = 'UHM' 

# UHM model with all the components fused together (i.e. ears, inner mouth, and teeth)
model_name = 'head_model_global_align'

model_file = open(models_folder + model_name + '.pkl', 'rb')
model_dict = pickle.load(model_file)
model_file.close()

In [ ]:
# turning coordinates system to be in millimeter units
scale_factor = 100

# get model parameteres
mean_shape = scale_factor*model_dict['Mean']
mean_shape_CCS = mean_shape.reshape(-1,3)
eigen_vec = model_dict['Eigenvectors']
eigen_vec_num =  model_dict['Eigenvectors'].shape[1]
eigen_val = model_dict['EigenValues']
trilist = model_dict['Trilist']
vertices_num = model_dict['Number_of_vertices']

In [ ]:
# load modules (landmarks and masks)
modules_folder = models_folder + '/Landmarks and masks/'

modules_to_load = ['68_land_idxs'] # EEG_10_20_full_model / '49_plus_ears_land_idxs' / '68_land_idxs'

landmarks = []
landmarks_names = []
landmarks_groups = []

for currnet_module_name in modules_to_load:
    module_file = open(modules_folder + currnet_module_name + '.pkl', 'rb')
    currnet_module = pickle.load(module_file)
    module_file.close()
    if currnet_module_name=='EEG_10_20':
        currnet_module_names = list(currnet_module.keys())
        currnet_module = np.asarray(list(currnet_module.values()))
    else:
        currnet_module_names = list(map(str, 1+np.arange(len(currnet_module))))
        
    landmarks.append(currnet_module)
    landmarks_names.append(currnet_module_names)
    landmarks_groups.append(np.arange(len(currnet_module)))

# turn list of lists into one list
landmarks = [item for items in landmarks for item in items]
landmarks_names = [item for items in landmarks_names for item in items]

num_of_landmarks = len(landmarks)

In [ ]:
module_names_to_remove = ['eyes_mask', 'teeth_mask', 'mouth_teeth_mask']
modules_to_remove = []

for module_name_to_remove in module_names_to_remove:
    module_file_to_remove = open(modules_folder + module_name_to_remove + '.pkl', 'rb')
    module_to_remove = pickle.load(module_file_to_remove)
    modules_to_remove.append(module_to_remove)
    module_file_to_remove.close()
    
mask_to_remove = np.logical_and(modules_to_remove[0], modules_to_remove[1], modules_to_remove[2])
mask_to_remove = ~mask_to_remove

## Lighter model including everything but eyes, teeth and inner mouth cavity

In [ ]:
# load model
light_models_folder = './datasets/TMS/3DMM/UHM_models/'
light_currnet_model = 'UHM' 

# UHM model with all the components fused together (i.e. ears, inner mouth, and teeth)
light_model_name = 'head_model_global_align_no_mouth_and_eyes'

light_model_file = open(light_models_folder + light_model_name + '.pkl', 'rb')
light_model_dict = pickle.load(light_model_file)
light_model_file.close()

In [ ]:
# turning cartesian coordinates system to be in millimeter units
light_scale_factor = 100

# get model parameteres
light_mean_shape = light_scale_factor*light_model_dict['Mean']
light_mean_shape_CCS = light_mean_shape.reshape(-1,3)
light_eigen_vec = light_model_dict['Eigenvectors']
light_eigen_vec_num =  light_model_dict['Eigenvectors'].shape[1]
light_eigen_val = light_model_dict['EigenValues']
light_trilist = light_model_dict['Trilist']
light_vertices_num = light_model_dict['Number_of_vertices']

In [ ]:
# load modules (landmarks and masks)
modules_folder = models_folder + '/Landmarks and masks/'

modules_to_load = ['EEG_10_20'] # EEG_10_20 / '49_plus_ears_land_idxs' / '68_land_idxs'

light_landmarks = []
light_landmarks_names = []
light_landmarks_groups = []

for currnet_module_name in modules_to_load:
    module_file = open(modules_folder + currnet_module_name + '.pkl', 'rb')
    currnet_module = pickle.load(module_file)
    module_file.close()
    if currnet_module_name=='EEG_10_20':
        currnet_module_names = list(currnet_module.keys())
        currnet_module = np.asarray(list(currnet_module.values()))
    else:
        currnet_module_names = list(map(str, 1+np.arange(len(currnet_module))))
        
    light_landmarks.append(currnet_module)
    light_landmarks_names.append(currnet_module_names)
    light_landmarks_groups.append(np.arange(len(currnet_module)))

# turn list of lists into one list
light_landmarks = [item for items in light_landmarks for item in items]
light_landmarks_names = [item for items in light_landmarks_names for item in items]

num_of_light_landmarks = len(light_landmarks)

## Matching landmark indices between models

In [ ]:
light_facial_landmarks = landmarks

for current_landmark_index, current_landmark_vertex in enumerate(landmarks):
    original_model_coordinates = mean_shape_CCS[current_landmark_vertex]
    new_model_vertex_diffs = np.linalg.norm(light_mean_shape_CCS-original_model_coordinates, axis=1)
    light_facial_landmarks[current_landmark_index] = np.argmin(new_model_vertex_diffs)

In [ ]:
light_nasion = 52241
light_inion = 36323

# Definitions

## Model Choosing

In [ ]:
choose_light_model = True

In [ ]:
if choose_light_model:
    landmarks = np.concatenate((light_landmarks, light_facial_landmarks, [light_nasion, light_inion]))
    landmarks_names = list(np.concatenate((light_landmarks_names, landmarks_names, ['nasion', 'inion'])))
    mean_shape = light_mean_shape
    mean_shape_CCS = light_mean_shape_CCS
    eigen_vec = light_eigen_vec
    eigen_vec_num =  light_eigen_vec_num
    eigen_val = light_eigen_val
    trilist = light_trilist
    vertices_num = light_vertices_num

## Parameters

In [ ]:
distinct_landmarks_names = np.array([37, 40, 43, 46, 49, 55, 31, 9])
rigid_facial_landmarks_names = np.array([37, 40, 43, 46, 28, 1, 17])

center_of_the_eyebrows = np.array([20, 25])
corners_of_the_eyebrows = np.array([18, 22, 23, 27])
corners_of_the_eyes = np.array([37, 40, 43, 46])
sides_of_the_face = np.array([1, 17])
nose_bone = np.array([28, 31])
lower_nose = np.array([32, 34, 36])
corners_of_the_mouth = np.array([49, 55])
chin = np.array([9])

facial_landmarks = np.concatenate((center_of_the_eyebrows, corners_of_the_eyebrows, corners_of_the_eyes, sides_of_the_face, nose_bone, lower_nose,
                                   corners_of_the_mouth, chin))
selected_facial_indices = np.sort(facial_landmarks+num_of_light_landmarks-1)
selected_facial_landmark_names = np.take(landmarks_names, selected_facial_indices)

selected_EEG_10_20_landmark_names = ['Fz', 'Pz', 'Cz', 'F3', 'F4', 'P3', 'P4', 'O1', 'O2']
selected_EEG_10_20_indices = []
for current_index, current_landmark_name in enumerate(selected_EEG_10_20_landmark_names):
    selected_EEG_10_20_indices.append(light_landmarks_names.index(current_landmark_name))
selected_EEG_10_20_indices = np.asarray(selected_EEG_10_20_indices)

In [ ]:
cranial_indices = [7, 13, 89, 90]

In [ ]:
input_landmarks = ['1', '2', '3', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27',
                   '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42',
                   '43', '44', '45', '46', '47', '48']
input_landmarks = [int(current_landmark) for current_landmark in input_landmarks]

In [ ]:
selected_indices = np.concatenate((selected_EEG_10_20_indices, 20+np.array(input_landmarks), cranial_indices))
selected_indices_names = np.take(landmarks_names, selected_indices)

In [ ]:
distances_EEG_10_20_landmark_names = ['Fp1', 'F7', 'T3', 'T5', 'O1', 'O2', 'T6', 'T4', 'F8', 'Fp2']
distances_EEG_10_20_indices = []
for current_index, current_landmark_name in enumerate(distances_EEG_10_20_landmark_names):
    distances_EEG_10_20_indices.append(light_landmarks_names.index(current_landmark_name))
distances_EEG_10_20_indices = np.asarray(distances_EEG_10_20_indices)

In [ ]:
distances_to_find = [landmarks[cranial_indices[:2]], #A1-A2
                     landmarks[cranial_indices[2:]], #nasion-inion
                     landmarks[distances_EEG_10_20_indices[:2]],
                     landmarks[distances_EEG_10_20_indices[1:3]],
                     landmarks[distances_EEG_10_20_indices[2:4]],
                     landmarks[distances_EEG_10_20_indices[3:5]],
                     landmarks[distances_EEG_10_20_indices[4:6]],
                     landmarks[distances_EEG_10_20_indices[5:7]],
                     landmarks[distances_EEG_10_20_indices[6:8]],
                     landmarks[distances_EEG_10_20_indices[7:9]],
                     landmarks[distances_EEG_10_20_indices[8:10]],
                     landmarks[np.array([distances_EEG_10_20_indices[-1], distances_EEG_10_20_indices[0]])]
                    ] 
distances_to_find_names = ['A1_A2', 'nasion_inion',
                           'Fp1_F7', 'F7_T3', 'T3_T5', 'T5_O1', 'O1_O2', 'O2_T6', 'T6_T4', 'T4_F8', 'F8_Fp2', 'Fp2_Fp1']

In [ ]:
# define parameteres
create_new = True
decimation = True
fixed_landmarks = True
calculate_rigid = False
num_of_instances = int(2.5e4)
num_digits_round = 10
num_of_landmarks = len(landmarks)
decimation_percentages = np.array([1])

## Functions

In [ ]:
# create new shape by adding deformations to the mean shape using the eigendecomposition
def get_new_shape_CCS(seed_number):
    np.random.seed(seed_number)
    # create eigen_val noised weights
    weights_coefficient_factor = np.round(0.35, num_digits_round) #np.round(np.random.uniform(0.2, 0.25), num_digits_round)
    eigen_val_weights = np.random.randn(eigen_vec_num, 1)*weights_coefficient_factor*scale_factor
    eigen_val_vec = eigen_val.reshape((eigen_val.shape[0], 1))

    eigen_vec_multipliers = np.multiply(eigen_val_weights, eigen_val_vec)
    added_deformation = eigen_vec @ eigen_vec_multipliers

    # add the added_deformation to the mean shape
    new_shape = mean_shape + added_deformation
    new_shape_CCS = new_shape.reshape(-1,3)

    return new_shape_CCS

In [ ]:
# get new head instance and its relevant features
def create_new_instance_coordinates_geodesic_distances_over_resolutions(current_instance_index):
    instance_EEG_10_20_geodesic_distances = np.zeros((len(decimation_percentages), len(distances_to_find)))
    instance_landmarks_coordinates_array = np.zeros((len(decimation_percentages), num_of_landmarks*3))

    new_shape_original_density_CCS = get_new_shape_CCS(current_instance_index)

    for decimation_index, decimation_percentage in enumerate(decimation_percentages):
        if decimation_index==0:
            landmarks_CCS = new_shape_original_density_CCS[landmarks, :]
            landmarks_CCS_array = landmarks_CCS.reshape(1, -1).squeeze()
            instance_landmarks_coordinates_array[decimation_index, :] = landmarks_CCS_array

            new_shape_original_density_trimesh = trimesh.Trimesh(vertices=new_shape_original_density_CCS, faces=trilist, process=True)

            current_shape_trimesh = new_shape_original_density_trimesh
        
            try:
                current_trimesh_landmarks_indices = get_trimesh_indices(current_shape_trimesh, new_shape_original_density_CCS, landmarks)
                current_indices = current_trimesh_landmarks_indices
            except:
                print(current_instance_index+" instance failed")
            
        instance_landmarks_coordinates_array[decimation_index, :] = np.array(current_shape_trimesh.vertices[current_indices]).ravel()

        current_shape_graph = trimesh.graph.vertex_adjacency_graph(current_shape_trimesh)
        current_shape_graph_weights = get_graph_weights(current_shape_graph, current_shape_trimesh)
        
    
    for i, current_distance in enumerate(distances_to_find):
        current_landmark_index_i = current_distance[0]
        current_landmark_index_j = current_distance[1]
        try:
            current_geodesic_distance = nx.dijkstra_path_length(current_shape_graph_weights,
                                                                current_landmark_index_i, 
                                                                current_landmark_index_j, 
                                                                weight='euclidean_distance')
        except:
            print(f"{current_instance_index}, {decimation_percentage}, cranial landmarks {current_landmark_index_i, current_landmark_index_j} geodesic distance failed")
            current_geodesic_distance = 0

        instance_EEG_10_20_geodesic_distances[0, i] = current_geodesic_distance
    
    return instance_landmarks_coordinates_array, instance_EEG_10_20_geodesic_distances

# Get mean shape model information

In [ ]:
mean_shape_trimesh = trimesh.Trimesh(vertices=mean_shape_CCS, faces=trilist, process=True)

if decimation==True and choose_light_model==False:
    mean_shape_trimesh.update_vertices(module_to_remove)

original_num_of_faces = mean_shape_trimesh.faces.shape[0]

In [ ]:
if choose_light_model==True:
    mean_trimesh_landmarks_indices = get_trimesh_indices(mean_shape_trimesh, mean_shape_CCS, landmarks)
else:
    mean_trimesh_landmarks_indices = get_trimesh_indices(mean_shape_trimesh, mean_shape_CCS[module_to_remove, :], landmarks)

mean_shape_graph = trimesh.graph.vertex_adjacency_graph(mean_shape_trimesh)
mean_shape_graph_weights = get_graph_weights(mean_shape_graph, mean_shape_trimesh)

In [ ]:
mean_shape_shortest_paths = []

if calculate_rigid==True:
    for i, current_landmark_index_i in enumerate(selected_EEG_10_20_indices):
        for j, current_landmark_index_j in enumerate(selected_facial_indices):
            current_geodesic_shortest_path = nx.dijkstra_path(mean_shape_graph_weights, 
                                                              mean_trimesh_landmarks_indices[current_landmark_index_i], 
                                                              mean_trimesh_landmarks_indices[current_landmark_index_j], 
                                                              weight='euclidean_distance')
            mean_shape_shortest_paths.append(current_geodesic_shortest_path)

# Create and save datasets

In [ ]:
landmarks_coordinates_array = np.zeros((num_of_instances, len(decimation_percentages), num_of_landmarks*3))
geodesic_distances_array = np.zeros((num_of_instances, len(decimation_percentages), len(distances_to_find)))

if calculate_rigid:
    rigid_geodesic_distances = np.zeros((num_of_instances, len(selected_EEG_10_20_indices)*len(selected_facial_indices)))

already_existing_instances = int(0e4)
pool = mp.Pool(mp.cpu_count()-4) #mp.Pool(mp.cpu_count()-3) / mp.Pool(6)
instances_results = []
start_time = time.time()
instances_results = pool.starmap(create_new_instance_coordinates_geodesic_distances_over_resolutions, 
                                 [(already_existing_instances+current_instance_index,) for current_instance_index in range(num_of_instances)])
end_time = time.time()
pool.close()

for current_instance in range(num_of_instances):
    landmarks_coordinates_array[current_instance, :, :] = instances_results[current_instance][0]
    geodesic_distances_array[current_instance, :, :] = instances_results[current_instance][1]
    #if calculate_rigid:
    #    rigid_geodesic_distances[current_instance, :] = instances_results[current_instance][2]
        
print((end_time-start_time)/3600)

In [ ]:
pool.close()

In [ ]:
landmarks_coordinates_array = landmarks_coordinates_array.squeeze()

In [ ]:
folder = "./datasets/TMS/3DMM/Head_instances/"
filetype = ".xlsx"
#filename = "32_light_landmarks_geodesic_distances_of_facial_landmarks_over_resolutions_and_coordinates_05_24"
filename = "dataset"
path = folder + filename + filetype

if decimation==False:
    if create_new==True:
        df_all_landmarks_names = np.repeat(landmarks_names, 3)
        df_coordinates_names = np.array(['x', 'y', 'z']*num_of_landmarks)
        landmarks_coordinates_df = pd.DataFrame(data=landmarks_coordinates_array, columns=
                                                pd.MultiIndex.from_tuples(zip(df_all_landmarks_names, df_coordinates_names)))

        geodesic_distances_df = pd.DataFrame(data=geodesic_distances_array, columns=
                                                pd.MultiIndex.from_tuples(zip(multiindex_up_names, multiindex_down_names)))

        rigid_geodesic_distances_df = pd.DataFrame(data=rigid_geodesic_distances, columns=
                                                   pd.MultiIndex.from_tuples(zip(multiindex_up_names, multiindex_down_names)))

        # create a excel writer object to write into multiple sheets
        with pd.ExcelWriter(path) as writer:
            landmarks_coordinates_df.round(num_digits_round).to_excel(writer, sheet_name='coordinates', index=1)
            geodesic_distances_df.round(num_digits_round).to_excel(writer, sheet_name='geodesic', index=2)
            rigid_geodesic_distances_df.round(num_digits_round).to_excel(writer, sheet_name='rigid_geodesic', index=3)
    else:
        # read facial features coordinates from an excel file as multiindex
        landmarks_coordinates_df = pd.read_excel(path, header=[0,1], index_col=0, sheet_name='coordinates')
        landmarks_coordinates = landmarks_coordinates_df.to_numpy()#[:, 1:]

        geodesic_distances_df = pd.read_excel(path, header=[0,1], index_col=0, sheet_name='geodesic')
        geodesic_distances = geodesic_distances_df.to_numpy()#[:, 1:]

        rigid_geodesic_distances_df = pd.read_excel(path, header=[0,1], index_col=0, sheet_name='rigid_geodesic')
        rigid_geodesic_distances = rigid_geodesic_distances_df.to_numpy()#[:, 1:]

else:
    landmarks_coordinates_df = []
    geodesic_distances_df = []
    
    if create_new==True:
        df_all_landmarks_names = np.repeat(landmarks_names, 3)
        df_coordinates_names = np.array(['x', 'y', 'z']*num_of_landmarks)
        
        for l in range(len(decimation_percentages)):
            landmarks_coordinates_df.append(pd.DataFrame(data=landmarks_coordinates_array, columns=
                                                    pd.MultiIndex.from_tuples(zip(df_all_landmarks_names, df_coordinates_names))))
            
            geodesic_distances_df.append(pd.DataFrame(data=geodesic_distances_array[:, l, :], columns=
                                                    #pd.MultiIndex.from_tuples(zip(multiindex_up_names, multiindex_down_names))))
                                                      distances_to_find_names))
            
        # create a excel writer object to write into multiple sheets
        with pd.ExcelWriter(path) as writer:
            added_index = 1
            
            if calculate_rigid:
                rigid_geodesic_distances_df.round(num_digits_round).to_excel(writer, sheet_name=('rigid_geodesic_'+str(decimation_percentages[0])), index=1)
                added_index+=1
            for l in range(len(decimation_percentages)):
                landmarks_coordinates_df[l].round(num_digits_round).to_excel(writer, sheet_name=('coordinates_'+str(decimation_percentages[l])), index=2*l+added_index)
                geodesic_distances_df[l].round(num_digits_round).to_excel(writer, sheet_name=('geodesic_'+str(decimation_percentages[l])), index=2*l+added_index+1)
                
    else:
        # read facial features coordinates from an excel file as multiindex
        if calculate_rigid:
            rigid_geodesic_distances_df = pd.read_excel(path, header=[0,1], index_col=0, sheet_name=('rigid_geodesic_'+str(decimation_percentages[0])))
            rigid_geodesic_distances = rigid_geodesic_distances_df.to_numpy()#[:, 1:]
        
        for l in range(len(decimation_percentages)):
            landmarks_coordinates_df[l] = pd.read_excel(path, header=[0,1], index_col=0, sheet_name=('coordinates_' + str(decimation_percentages[l])))
            landmarks_coordinates[l] = landmarks_coordinates_df[l].to_numpy()#[:, 1:]
            
            geodesic_distances_df[l] = pd.read_excel(path, header=[0,1], index_col=0, sheet_name=('geodesic_' + str(decimation_percentages[l])))
            geodesic_distances[l] = geodesic_distances_df[l].to_numpy()#[:, 1:]

# Merge datasets

In [ ]:
# merge dataframes
if 1:
    from datetime import datetime
    timestamp_string = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    timestamp_string = timestamp_string.replace('_2022_', '_22_')
    
    folder = "./datasets/TMS/3DMM/Head_instances/"
    filetype = ".xlsx"
    
    merged_filename = "coordintaes_geodesic_distances_30_08"
    merged_path = folder + merged_filename + filetype
    
    first_filename = "81028"
    first_path = folder + first_filename + filetype
    
    second_filename = "08_29"
    second_path = folder + second_filename + filetype

    first_landmarks_coordinates_df = []
    first_geodesic_distances = []

    second_landmarks_coordinates_df = []
    second_geodesic_distances = []

    merged_landmarks_coordinates_df = []
    merged_geodesic_distances_df = []

    for l in range(len(decimation_percentages)):
        first_landmarks_coordinates_df.append(pd.read_excel(first_path, header=[0,1], index_col=0, sheet_name=('coordinates_' + str(decimation_percentages[l]))))
        current_first_geodesic_distances_df = pd.read_excel(first_path, header=[0,1], index_col=0, sheet_name=('geodesic_' + str(decimation_percentages[l])))
        first_geodesic_distances.append(np.asarray(current_first_geodesic_distances_df))

        first_num_of_instances = first_landmarks_coordinates_df[l].shape[0]

        second_landmarks_coordinates_df.append(pd.read_excel(second_path, header=[0,1], index_col=0, sheet_name=('coordinates_' + str(decimation_percentages[l]))))
        current_second_geodesic_distances_df = pd.read_excel(second_path, header=[0,1], index_col=0, sheet_name=('geodesic_' + str(decimation_percentages[l])))
        second_geodesic_distances.append(np.asarray(current_second_geodesic_distances_df))

        second_num_of_instances = second_landmarks_coordinates_df[l].shape[0]

        merged_landmarks_coordinates_df.append(pd.concat([first_landmarks_coordinates_df[l], second_landmarks_coordinates_df[l]]))
        merged_landmarks_coordinates_df[l].index = np.arange(0, first_num_of_instances+second_num_of_instances)

        current_merged_geodesic_distances_df = pd.DataFrame(data=np.concatenate((
            first_geodesic_distances[l], second_geodesic_distances[l])), columns=pd.MultiIndex.from_tuples(
            zip(multiindex_up_names, multiindex_down_names)))
        merged_geodesic_distances_df.append(current_merged_geodesic_distances_df)

    with pd.ExcelWriter(merged_path) as writer:
        added_index = 1

        if calculate_rigid:
            rigid_geodesic_distances_df.round(num_digits_round).to_excel(writer, sheet_name=('rigid_geodesic_'+str(decimation_percentages[0])), index=1)
            added_index+=1
        for l in range(len(decimation_percentages)):
            merged_landmarks_coordinates_df[l].round(num_digits_round).to_excel(writer, sheet_name=('coordinates_'+str(decimation_percentages[l])), index=2*l+added_index)
            merged_geodesic_distances_df[l].round(num_digits_round).to_excel(writer, sheet_name=('geodesic_'+str(decimation_percentages[l])), index=2*l+added_index+1)
